# ch04/gradient_1d.py

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import sys, os
sys.path.append('/content/drive/MyDrive/Colab Notebooks/밑바닥부터시작하는')
sys.path.append(os.pardir)
sys.path.append('/content/drive/MyDrive/Colab Notebooks/밑바닥부터시작하는/ch04')
sys.path.append('/content/drive/MyDrive/Colab Notebooks/밑바닥부터시작하는/common')

Mounted at /content/drive


In [ ]:
## 손실함수 ##

# #오차제곱합
# y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]
# t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]

# def sum_squares_error(y, t):
#   return 0.5 * np.sum((y-t)**2)

# t = [0, 0, 1, 0, 0, 0, 0, 0, 0]

# #예1: '2'일 확률이 가장 높다고 추정함 (0.6)
# y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]
# sum_squares_error(np.array(y), np.array(t)) #0.097500000000000031

# #예2: '7'일 확률이 가장 높다고 추정함 (0.6)
# y = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0]
# sum_squares_error(np.array(y), np.array(t)) #0.59750000000000003


# def cross_entropy_error(y, t):
#   delta = 1e-7 # 0일때 -무한대가 되지 않기 위해 작은 값을 더함
#   return -np.sum(t* np.log(y + delta))

# t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
# y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]
# cross_entropy_error(np.array(y), np.array(t)) #0.51082545709933802

# y = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0]
# cross_entropy_error(np.array(y), np.array(t)) #2,3025840929945458

# ## 미니배치 학습 ##
# import sys, os
# sys.path.append(os.pardir)
# import numpy as np
# from mnist import load_mnist

# (x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)
# print(x_train.shape) # (60000, 784)
# print(t_train.shape) # (60000, 10)

train_size = x_train.shape[0]
batch_size = 10
batch_mask = np.random.choice(train_size, batch_size) # 범위 중 무작위 10개 추출
x_batch = x_train[batch_mask]
t_batch = t_train[batch_mask]

#np.random.choice(60000, 10) #array([ 3070, 49842, 13487, 11604, 50348, 17294,  5694, 35492, 56758,58402])


In [ ]:
## (배치용) 교차 엔트로피 오차 구현하기 ##
def cross_entropy_error(y, t):
  if y.ndim == 1:
    t = t.reshape(1, t.size)
    y = y.reshape(1, y.size)
  batch_size = y.shape[0]
  return -np.sum(t* np.log(y+ 1e-7)) / batch_size

def cross_entropy_error(y, t):
  if y.ndim == 1:
    t = t.reshape(1, t.size)
    y = y.reshape(1, y.size)
  batch_size = y.shape[0]
  return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size


t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0 ]
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]
cross_entropy_error(np.array(y), np.array(t)) #0.510825457099338
y = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0]
cross_entropy_error(np.array(y), np.array(t)) #2.302584092994546

수치미분

In [5]:
import numpy as np
import matplotlib.pylab as plt

## 나쁜 구현 예 ##
def numerical_diff(f, x):
    h = 1e-50
    return (f(x + h) - f(x)) / h

# 반올림 오차문제(너무작은값)
np.float32(1e-50) #0.0

## 중심차분, 중앙차분 ##
def numerical_diff(f, x):
    h = 1e-4 # 0.0001
    return (f(x+h) - f(x-h)) / (2*h)


## 수치미분의 예 ##
def function_1(x):
    return 0.01*x**2 + 0.1*x

#접선구하는 함수
def tangent_line(f, x):
    d = numerical_diff(f, x)
    print(d)
    y = f(x) - d*x
    return lambda t: d*t + y

x = np.arange(0.0, 20.0, 0.1)  # 0에서 20까지 0.1 간격의 배열 X를 만든다(20은 미포함)
y = function_1(x)
# plt.xlabel("x")
# plt.ylabel("f(x)")
# plt.plot(x, y)
# plt.show()

tf = tangent_line(function_1, 5)
y2 = tf(x)

plt.plot(x, y)
plt.plot(x, y2)
plt.show()

numerical_diff(function_1, 5)
numerical_diff(function_1, 10)

## 편미분 ##
def function_2(x):
  return x[0]**2 + x[1]**2  #또는 return np.sum(x**2)

# # Q1. x0 = 3, x1 = 4일때, x0에 대한  편미분
def function_tmp1(x0):
  return x0**2 + 4.0**2.0

numerical_diff(function_tmp1, 3.0)  # 6.00000000000378

# Q2. x0 = 3, x1 = 4일때, x1에 대한  편미분
# def function_tmp2(x1):
#   return 3.0**2.0 + x1*x1

# numerical_diff(function_tmp2, 4.0) # 7.999999999999119

6.00000000000378

# ch04/gradient_2d.py

기울기

In [7]:
def numerical_gradient(f, x):
  h = 1e-4 # 0,0001
  grad = np.zeros_like(x) # x와 형상이 같은 배열을 생성
  for idx in range(x.size):
    tmp_val = x[idx] # f(x+h)
    x[idx] = tmp_val + h
    fxh1= f(x)
    #f(x-h) 계산
    x[idx] = tmp_val - h
    fxh2 = f(x)

    grad[idx] = (fxh1-fxh2) / (2*h)
    x[idx] = tmp_val #값 복원
  return grad

numerical_gradient(function_2, np.array([3.0, 4.0]))
numerical_gradient(function_2, np.array([0.0, 2.0]))
numerical_gradient(function_2, np.array([3.0, 0.0])) #array([6., 0.])


array([6., 0.])

# ch04/gradient_method.py

In [ ]:
import numpy as np
import matplotlib.pylab as plt

## 경사 하강법 ##
def gradient_descent(f, init_x, lr=0.01, step_num=100):
    x = init_x
    x_history = []

    for i in range(step_num):
        x_history.append( x.copy() )

        grad = numerical_gradient(f, x)
        x -= lr * grad

    return x, np.array(x_history)

#Q1 : 경사법으로 f(xo,x1)=x0**2 + x1**2 의 최솟값을 구하라.
def function_2(x):
    return x[0]**2 + x[1]**2

init_x = np.array([-3.0, 4.0])
gradient_descent(function_2, init_x=init_x, lr=0.1, step_num=100) #array([ -6,11110793e-10, 8.14814391e-10])


# #학습률이 너무 큰 예: 1r=10.0
# init_x = np.array([-3.0, 4.0])
# gradient_descent(function_2, init_x=init_x, lr=10.0, step_num=100) #array([ -2.58983747e+13, -1.29524862e+12])

# #학습률이 너무 작은 예: lr=1e-10
# init_x = np.array([-3.0, 4.0])
# gradient_descent (function_2, init_x=init_x, lr=1e-10, step_num=100) #array([-2.99999994, 3.99999992])

# # 그래프
# init_x = np.array([-3.0, 4.0])
# x, x_history = gradient_descent(function_2, init_x, lr=0.1, step_num=20)

# plt.plot([-5, 5], [0, 0], '--b')
# plt.plot([0, 0], [-5, 5], '--b')
# plt.plot(x_history[:, 0], x_history[:, 1], 'o')

# plt.xlim(-3.5, 3.5)
# plt.ylim(-4.5, 4.5)
# plt.xlabel("X0")
# plt.ylabel("X1")
# plt.show()

# ch04/gradient_simplenet.py

In [ ]:
##신경망에서의 기울기 ##
import sys, os
sys.path.append(os.pardir)
import numpy as np
from common.functions import softmax, cross_entropy_error
from common.gradient import numerical_gradient


class simpleNet:
    def __init__(self):
        self.W = np.random.randn(2,3) #정규분포로 초기화

    def predict(self, x):
        return np.dot(x, self.W)

    def loss(self, x, t):
        z = self.predict(x)
        y = softmax(z)
        loss = cross_entropy_error(y, t)

        return loss

net = simpleNet()
#print(net.W) #가중치 매개변수 [[-1.02616074 -0.30301655 -0.63530905] [ 1.15123149 -0.53630072 -1.5930804 ]]

x = np.array([0.6, 0.9])
#p = net.predict(x)
# print(p) #[ 0.04701656 -1.78292584  0.34509516]
#np.argmax(p) #최대값의 idx #0

t = np.array([0, 0, 1]) #정답레이블
#net.loss(x, t) #2.1237823839422174

def f(W):
  return net.loss(x,t)
dW = numerical_gradient(f, net.W)
print(dW) #[[ 0.45987683  0.06436874 -0.52424558] [ 0.68981525  0.09655312 -0.78636837]]

# net = simpleNet()
#상위함수 lamda로
f = lambda w: net.loss(x, t)
dW = numerical_gradient(f, net.W)

# print(dW)

[[ 0.06858494  0.37487234 -0.44345728]
 [ 0.1028774   0.56230851 -0.66518592]]


# ch04/two_layer_net.py

학습 알고리즘 구현하기
> 전제
  1단계-미니배치
  2단계-기울기 산출
  3단계-매개변수 갱신
  4단계 반복





In [ ]:
## 2층 신경망 Class 구현하기 ##

import sys, os
sys.path.append(os.pardir)
from common.functions import *
from common.gradient import numerical_gradient
import numpy as np


class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)

    def predict(self, x):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']

        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)

        return y

    # 손실함수, x:입력데이터, t:정답 레이블
    def loss(self, x, t):
        y = self.predict(x)

        return cross_entropy_error(y, t)

    #정확도
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        t = np.argmax(t, axis=1)

        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy

    # 가중치 매개변수의 기울기. x:입력데이터, t:정답레이블
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)

        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])

        return grads

    # def gradient(self, x, t):
    #     W1, W2 = self.params['W1'], self.params['W2']
    #     b1, b2 = self.params['b1'], self.params['b2']
    #     grads = {}

    #     batch_num = x.shape[0]

    #     # forward
    #     a1 = np.dot(x, W1) + b1
    #     z1 = sigmoid(a1)
    #     a2 = np.dot(z1, W2) + b2
    #     y = softmax(a2)

    #     # backward
    #     dy = (y - t) / batch_num
    #     grads['W2'] = np.dot(z1.T, dy)
    #     grads['b2'] = np.sum(dy, axis=0)

    #     dz1 = np.dot(dy, W2.T)
    #     da1 = sigmoid_grad(a1) * dz1
    #     grads['W1'] = np.dot(x.T, da1)
    #     grads['b1'] = np.sum(da1, axis=0)

    #     return grads

#예
net = TwoLayerNet(input_size=784, hidden_size=100, output_size=10)
net.params['W1'].shape
net.params['b1'].shape
net.params['W2'].shape
net.params['b2'].shape #(10,)

x = np.random.rand(100,784) #더미'입력'데이터(100장분량)
y = net.predict(x)
t= np.random.rand(100,10) #더미'정답'데이터(100장분량)

grads = net.numerical_gradient(x,t) #기울기 계산 (오래걸림 주의)
net.params['W1'].shape
net.params['b1'].shape
net.params['W2'].shape
net.params['b2'].shape #(10,)


(10,)

# ch04/train_neuralnet.py

In [ ]:
#미니배치 학습 구현하기
import sys, os
sys.path.append(os.pardir)

import numpy as np
import matplotlib.pyplot as plt
from dataset.mnist import load_mnist
from two_layer_net import TwoLayerNet


(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)
network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

# 하이퍼파라미터
iters_num = 10000  #반복 획수를 적절히 설정한다
train_size = x_train.shape[0]
batch_size = 100 #미니배치크기
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
    #미니배치 획득
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]

    # 기울기 계산
    #grad = network.numerical_gradient(x_batch, t_batch)
    grad = network.gradient(x_batch, t_batch) #성능 개선판!

    # 매개변수 갱신
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]

    # 학습 경과 기록
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)

    # 1에폭당 정확도 계산
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print("train acc, test acc | " + str(train_acc) + ", " + str(test_acc))

AttributeError: 'TwoLayerNet' object has no attribute 'gradient'